# Explorando tendencias de mercado de Airbnb

In [0]:
from pyspark.sql import (
    SparkSession,
    types,
    functions as F,
)

spark = (
    SparkSession
    .builder
    .appName('airbnb')
    .getOrCreate()
)

## Cargar archivos de diferentes formatos

### Cargar archivo TSV, CSV, Excel

In [0]:

df_last_review = spark.read.option("delimiter", "\t").csv("dbfs:/FileStore/airbnb_last_review.tsv", header=True, inferSchema=True)
df_last_review.show()


+----------+----------------+---------------+
|listing_id|       host_name|    last_review|
+----------+----------------+---------------+
|      2595|        Jennifer|    May 21 2019|
|      3831|     LisaRoxanne|   July 05 2019|
|      5099|           Chris|   June 22 2019|
|      5178|        Shunichi|   June 24 2019|
|      5238|             Ben|   June 09 2019|
|      5295|            Lena|   June 22 2019|
|      5441|            Kate|   June 23 2019|
|      5803|          Laurie|   June 24 2019|
|      6021|         Claudio|   July 05 2019|
|      6848|   Allen & Irina|   June 29 2019|
|      7097|            Jane|   June 28 2019|
|      7322|            Doti|   July 01 2019|
|      7726|Adam And Charity|   June 22 2019|
|      8024|           Lisel|   July 01 2019|
|      8025|           Lisel|January 01 2019|
|      8110|           Lisel|   July 02 2019|
|      8490|        Nathalie|   June 19 2019|
|      8505|         Gregory|   June 23 2019|
|      9518|            Shon|   Ju

In [0]:
df_price = spark.read.csv("dbfs:/FileStore/airbnb_price.csv", header=True, inferSchema=True)
df_price.show()

+----------+-----------+--------------------+
|listing_id|      price|         nbhood_full|
+----------+-----------+--------------------+
|      2595|225 dollars|  Manhattan, Midtown|
|      3831| 89 dollars|Brooklyn, Clinton...|
|      5099|200 dollars|Manhattan, Murray...|
|      5178| 79 dollars|Manhattan, Hell's...|
|      5238|150 dollars|Manhattan, Chinatown|
|      5295|135 dollars|Manhattan, Upper ...|
|      5441| 85 dollars|Manhattan, Hell's...|
|      5803| 89 dollars|Brooklyn, South S...|
|      6021| 85 dollars|Manhattan, Upper ...|
|      6848|140 dollars|Brooklyn, William...|
|      7097|215 dollars|Brooklyn, Fort Gr...|
|      7322|140 dollars|  Manhattan, Chelsea|
|      7726| 99 dollars|Brooklyn, Crown H...|
|      8024|130 dollars|Brooklyn, Park Slope|
|      8025| 80 dollars|Brooklyn, Park Slope|
|      8110|110 dollars|Brooklyn, Park Slope|
|      8490|120 dollars|Brooklyn, Bedford...|
|      8505| 60 dollars|Brooklyn, Windsor...|
|      9518| 44 dollars|   Manhatt

In [0]:
# Leer el archivo CSV con comillas
df_room_type_raw = spark.read.csv("/FileStore/airbnb_room_type.csv", header=True, inferSchema=True)

# Limpiar las comillas de las columnas usando `withColumn` y `regexp_replace`
df_room_type = df_room_type_raw.select(
    [F.regexp_replace(F.col(col), '"', '').alias(col) for col in df_room_type_raw.columns]
)

# Mostrar las primeras filas del DataFrame
df_room_type.show()

+----------+--------------------+---------------+
|listing_id|         description|      room_type|
+----------+--------------------+---------------+
|      2595|Skylit Midtown Ca...|Entire home/apt|
|      3831|Cozy Entire Floor...|Entire home/apt|
|      5099|Large Cozy 1 BR A...|Entire home/apt|
|      5178|Large Furnished R...|   private room|
|      5238|Cute & Cozy Lower...|Entire home/apt|
|      5295|Beautiful 1br on ...|Entire home/apt|
|      5441|Central Manhattan...|   Private room|
|      5803|Lovely Room 1, Ga...|   Private room|
|      6021|Wonderful Guest B...|   Private room|
|      6848|Only 2 stops to M...|entire home/apt|
|      7097|Perfect for Your ...|Entire home/apt|
|      7322|     Chelsea Perfect|   Private room|
|      7726|Hip Historic Brow...|Entire home/apt|
|      8024|CBG CtyBGd HelpsH...|   PRIVATE ROOM|
|      8025|CBG Helps Haiti R...|   PRIVATE ROOM|
|      8110|CBG Helps Haiti R...|   Private room|
|      8490|MAISON DES SIRENE...|Entire home/apt|


## ¿Cuáles son las fechas de las primeras y las últimas reseñas? Almacene estos valores como dos variables independientes con los nombres que prefiera.

Lo primero que he hecho ha sido formatear la fecha, y como no me estaba funcionando nada puees lo he hehco de una manera un poco mas manual con un minidiccionario para el mes y luego remplazanod los espacios por las barras

In [0]:
from pyspark.sql.functions import col, regexp_replace

# Crear un diccionario para reemplazar los nombres de los meses por números
month_dict = {
    "January": "01", "February": "02", "March": "03", "April": "04", "May": "05", "June": "06",
    "July": "07", "August": "08", "September": "09", "October": "10", "November": "11", "December": "12"
}

# Reemplazar los nombres de los meses en la columna 'last_review' por el número de mes correspondiente
for month, num in month_dict.items():
    df_last_review = df_last_review.withColumn("last_review", regexp_replace("last_review", month, num))

# Reemplazar el espacio entre los componentes de la fecha por un guion
df_last_review = df_last_review.withColumn("last_review", regexp_replace("last_review", " ", "-"))

# Mostrar el resultado final
df_last_review.show(truncate=False)

+----------+----------------+-----------+
|listing_id|host_name       |last_review|
+----------+----------------+-----------+
|2595      |Jennifer        |05-21-2019 |
|3831      |LisaRoxanne     |07-05-2019 |
|5099      |Chris           |06-22-2019 |
|5178      |Shunichi        |06-24-2019 |
|5238      |Ben             |06-09-2019 |
|5295      |Lena            |06-22-2019 |
|5441      |Kate            |06-23-2019 |
|5803      |Laurie          |06-24-2019 |
|6021      |Claudio         |07-05-2019 |
|6848      |Allen & Irina   |06-29-2019 |
|7097      |Jane            |06-28-2019 |
|7322      |Doti            |07-01-2019 |
|7726      |Adam And Charity|06-22-2019 |
|8024      |Lisel           |07-01-2019 |
|8025      |Lisel           |01-01-2019 |
|8110      |Lisel           |07-02-2019 |
|8490      |Nathalie        |06-19-2019 |
|8505      |Gregory         |06-23-2019 |
|9518      |Shon            |06-15-2019 |
|9657      |Dana            |04-19-2019 |
+----------+----------------+-----

Luego ya como tienen el formato aplicado solo tenemos que hacer un minimo y un maximo

In [0]:
from pyspark.sql.functions import min, max

# Obtener la fecha de la primera reseña (mínima) y la última reseña (máxima)
first_review_date = df_last_review.select(min("last_review")).collect()[0][0]
last_review_date = df_last_review.select(max("last_review")).collect()[0][0]

# Mostrar los resultados
print(f"Fecha de la primera reseña: {first_review_date}")
print(f"Fecha de la última reseña: {last_review_date}")


Fecha de la primera reseña: 01-01-2019
Fecha de la última reseña: 07-09-2019


## ¿Cuántos de los anuncios son habitaciones privadas? Guárdelo en cualquier variable.


> Aqui tambien hay un problema y esq el termino de "private room" cada vez esta escrito de una manera, entonces lo que pasa es que hay que convertirlo todo a minusculas y luego ya buscar eso en la columna de `room_type`

In [0]:
from pyspark.sql import functions as F

# Convertir la columna 'room_type' a minúsculas
df_room_type_clean = df_room_type.withColumn("room_type_lower", F.lower(F.col("room_type")))

# Filtrar los anuncios que son habitaciones privadas
df_private_rooms = df_room_type_clean.filter(F.col("room_type_lower").contains("private room"))

# Contar el número de anuncios de habitaciones privadas
num_private_rooms = df_private_rooms.count()

# Guardar el resultado en una variable
num_private_rooms

# Mostrar el resultado
print(f"Número de anuncios de habitaciones privadas: {num_private_rooms}")

Número de anuncios de habitaciones privadas: 1942


## ¿Cuál es el precio promedio de los anuncios? Redondee a los dos decimales más cercanos y guárdelo en una variable.

> Aqui nos encontramos el problema de que la tabla no es numerica sino que tiene puesto "dollars" en todos los campos, asiq si no queremos que salga nulo hay que limpiarlos tambien antes eliminando esa palabra

In [0]:
from pyspark.sql import functions as F

# Limpiar la columna 'price' eliminando la palabra 'dollars' y cualquier otro texto
df_price_clean = df_price.withColumn("price", F.regexp_replace(F.col("price"), " dollars", "").cast("double"))

# Eliminar filas con valores nulos en la columna 'price'
df_price_clean = df_price_clean.filter(F.col("price").isNotNull())

# Calcular el precio promedio de los anuncios
average_price = df_price_clean.agg(F.avg("price").alias("average_price")).collect()[0]["average_price"]

# Verificar si el resultado es None
if average_price is not None:
    average_price_rounded = round(average_price, 2)
else:
    average_price_rounded = None

# Imprimir el resultado
print(f"El precio promedio de los anuncios es: {average_price_rounded}")


El precio promedio de los anuncios es: 141.78


## Combine las nuevas variables en un DataFrame llamado review_dates con cuatro columnas en el siguiente orden: first_reviewed, last_reviewed, nb_private_rooms y avg_price. El DataFrame solo debe contener una fila de valores.

Aqui yo entiendo que las nuevas variables de las que me habla son: 
- first_review_date 
- last_review_date
- num_private_rooms
- average_price_rounded

In [0]:
from pyspark.sql import Row

# Crear un Row con los valores de las variables
review_dates_row = Row(
    first_reviewed=first_review_date,
    last_reviewed=last_review_date,
    nb_private_rooms=num_private_rooms,
    avg_price=average_price_rounded
)

# Crear un DataFrame con esta fila
review_dates = spark.createDataFrame([review_dates_row])

# Mostrar el DataFrame resultante
review_dates.show()


+--------------+-------------+----------------+---------+
|first_reviewed|last_reviewed|nb_private_rooms|avg_price|
+--------------+-------------+----------------+---------+
|    01-01-2019|   07-09-2019|            1942|   141.78|
+--------------+-------------+----------------+---------+

